<h1>Chapter 13. Interfaces, Protocols and ABC</h1>

Object-oriented programming is about interfaces. If you want to understand what a type does in Python, find out what methods it provides (its interface).

<h2>Typing Types</h2>

<p><b>Duck Typing</b>:<br>
&emsp;&emsp;Definition: A programming style where an object's suitability is determined by the presence of certain methods and properties rather than the object's actual type.<br>
&emsp;&emsp;Example: If an object has a <code>quack</code> method, it can be treated as a "duck" even if it doesn't inherit from a specific duck class.</p>
<p><b>Goose Typing</b>:<br>
&emsp;&emsp;Definition: A variation of duck typing that also considers the types of the object's methods and attributes, ensuring they match expected types.<br>
&emsp;&emsp;Example: An object not only needs to have certain methods and attributes, but they must also adhere to specific types, e.g., method <code>quack</code> must take and return specific types.</p>
<p><b>Static Typing</b>:<br>
&emsp;&emsp;Definition: A programming style where variable types are explicitly declared and determined at compile-time.<br>
&emsp;&emsp;Example: In languages like Java, you must declare a variable's type (e.g., `int x = 5;`), and it cannot change.</p>
<p><b>Static Duck Typing</b>:<br>
&emsp;&emsp;Definition: A hybrid approach that combines static typing with duck typing.<br>
&emsp;&emsp;Example: A function can specify that it expects an object that has a particular method signature, allowing type-checking at compile time while maintaining flexibility in what kinds of objects can be passed in.</p>

<h2>Two Types of Protocols</h2>

<p><b>Dynamic Protocol</b>:<br>
&emsp;&emsp;Definition: In Python, dynamic protocols are abstract base classes (ABCs) that define a set of methods and attributes that an object must implement to be considered compliant with the protocol. These protocols are typically defined in the <code>collections</code> or <code>typing</code> module.<br>
&emsp;&emsp;Example: The <code>Sequence</code> protocol from the <code>collections</code> module defines a set of methods (<code>__getitem__</code>, <code>__len__</code>, etc.) that an object must implement to be considered a sequence.</p>

<p><b>Static Protocol</b>:<br>
&emsp;&emsp;Definition: Python static protocols are similar to dynamic protocols but make use of the <code>typing.Protocol</code> class from the <code>typing</code> module. These protocols use type hints to define the expected interface (methods and attributes) of a class at static type-checking time. Static protocols provide better tooling support and more explicit type checking.<br>
&emsp;&emsp;Example: A protocol can define that an object must have certain methods and attributes. For instance, a <code>Printable</code> protocol may require an object to implement a <code>print</code> method.</p>

<h2>Monkey Patching as a Means of Implementing a Protocol at Runtime</h2>

Monkey patching is a technique in which the behavior of existing code (classes, methods, functions, etc.) is modified at runtime. This can involve adding, modifying, or replacing existing methods or attributes.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
from random import shuffle

try:
    deck = FrenchDeck()
    shuffle(deck)
except TypeError as e:
    print(e.__repr__())

TypeError("'FrenchDeck' object does not support item assignment")


Monkey patching of the `FrenchDeck` class to make it modifiable and compatible with the `rundom.shuffle` function

In [3]:
def set_card(deck, position, card):
    """
    Function that accepts deck, position, and card arguments.
    """
    deck._cards[position] = card

Assigning a function to the `__setitem__` attribute of the `FrenchDeck` class

In [4]:
FrenchDeck.__setitem__ = set_card

In [5]:
shuffle(deck)
deck[:5]

[Card(rank='9', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='5', suit='hearts'),
 Card(rank='5', suit='diamonds'),
 Card(rank='K', suit='diamonds')]

<h2>ABC Subclass Creation</h2>

`abc.MutableSequence` is an abstract base class (ABC). It serves as a blueprint for creating mutable sequence data types, such as lists, that can be modified after creation.<br>
Purpose: Provides a standard set of methods that subclasses must implement to be considered mutable sequences.

In [6]:
from collections import abc, namedtuple

Card = namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init_(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.rank]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

In [7]:
try:
    deck = FrenchDeck2()
except TypeError as e:
    print(e.__repr__())

TypeError("Can't instantiate abstract class FrenchDeck2 without an implementation for abstract methods '__delitem__', 'insert'")


To create a subclass of `MutableSequence`, it is necessary to implement the `__delitem__`, `insert` abstract methods defined in this ABC.

In [8]:
from collections import abc, namedtuple

Card = namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init_(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.rank]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

<h2>Definition and Use of ABC</h2>

`Tombola` is an `ABC` with two abstract and two concrete methods

In [9]:
from abc import ABC, abstractmethod


class Tombola(ABC):

    @abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abstractmethod
    def pick(self):
        """
        Remove item at random, returning it.

        This method should raise 'LookupError' when the instance is empty.
        """

    def loaded(self):
        """Return 'True' if there is at least 1 item, 'False' otherwise."""

        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""

        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [10]:
class Fake(Tombola):
    def pick(self):
        return 13

In [11]:
try:
    f = Fake()
except TypeError as e:
    print(e.__repr__())

TypeError("Can't instantiate abstract class Fake without an implementation for abstract method 'load'")


<h3>ABC Syntactic Details</h3>

If `abstractmethod()` is used in conjunction with other method descriptors, it must be the innermost one. There should be no other decorators between `@abstractmethod` and `def`.

In [12]:
class MyABC(ABC):
    @classmethod
    @abstractmethod
    def an_abstract_classmethod(cls):
        pass

<h3>Creating ABC Subclasses</h3>

`BingoCage` is a specific subclass of `Tombola`

In [13]:
import random


class BingoCage(Tombola):

    def __init__(self, items):
        # Use SystemRandom for cryptographically secure randomization
        self._randomizer = random.SystemRandom()
        self._items = []
        # Initialize the items by calling the load() method
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        # Use the .shuffle() method of SystemRandom instance
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

`LotteryBlower` is a concrete subclass that overrides the `inspect` and `loaded` methods of ABC `Tombola`

In [14]:
class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        """
        The random.randrange() function raises ValueError exception
        if the range is empty, intercepts it and raises a LookupError exception
        instead, preserving ABC Tombola compatibility.

        Otherwise, a random element is selected from self._balls
        """
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

<h3>Virtual Subclass <code>Tombola</code></h3>

ABC virtual classes use the `register` function to declare a class as a virtual subclass, meaning it is recognized as a subclass without inheriting directly. This allows for flexible type checking and ensures the virtual subclass adheres to the ABC's interface.

In [15]:
# TomboList is registered as a virtual subclass of Tombola
@Tombola.register
class TomboList(list):  # TomboList expands the list

    def pick(self):
        """
        TomboList inherits the __bool__ method from list,
        which returns True if the list is not empty.

        The pick method calls the self.pop method inherited from list
        passing the index of a random item.
        """
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    # TomboList.load is the same as list.extend
    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(self)

In [16]:
issubclass(TomboList, Tombola)

True

In [17]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [18]:
TomboList.__mro__

(__main__.TomboList, list, object)